In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import torch
import os
import pickle
from itertools import product
from sklearn.metrics import mean_squared_error

from torch.utils.tensorboard import SummaryWriter

np.random.seed(101)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/ML Project CS6140/Assignment 4')

In [ ]:
ratings_df = pd.read_csv('ml-latest-small/ratings.csv')
movies_df = pd.read_csv('ml-latest-small/movies.csv')

In [ ]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
ratings_df[ratings_df['rating'] == 0.0]

,userId,movieId,rating,timestamp


In [ ]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
data = pd.merge(ratings_df, movies_df, on= ['movieId'])

In [ ]:
data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [ ]:
X_matrix_data = pd.pivot_table(data, values='rating', index='userId', columns='movieId')

In [ ]:
X_matrix_data

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,2.5,NaN,NaN,NaN,3.5,NaN,4.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,4.5,NaN,NaN,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,2.0,NaN,3.5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.5,3.5,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
X_matrix = X_matrix_data.values

In [ ]:
X_matrix_no_nan = np.nan_to_num(X_matrix, nan=0.0)

In [ ]:
unravel_unique_cats = list()
for i in unique_cats:
    unravel_unique_cats += i.split('|')

In [ ]:
unique_genres = set(unravel_unique_cats)

In [ ]:
k_values = [2, 5, 10, 15, len(unique_genres)]
lambda_reg_values = [1, 0.1, 0.01]
f_name = list()

for k, lambda_reg in product(k_values, lambda_reg_values):
    f_name.append("k="+str(k)+"_lambda="+str(lambda_reg)+".pkl")

In [ ]:
test_index_df = pd.read_csv('test_index_data.csv')

In [ ]:
X_matrix_data = X_matrix_data.fillna(0)
for f_num in range(len(f_name)):
    try:
      with open(f_name[f_num], 'rb') as file:
          d = pickle.load(file)
    except:
        print("No File found ")
        continue
    v_matrix = d['Final Movie matrix']
    u_matrix = d['Final User matrix']
    X_hat = np.dot(u_matrix, v_matrix.T)
    X_hat_df = pd.DataFrame(X_hat)
    X_hat_df.columns = X_matrix_data.columns
    X_hat_df.index = X_matrix_data.index
    analysis = list()
    for row in range(len(test_index_df)):
        u, m, true_rating = test_index_df.iloc[row][['userId', 'movieId', 'rating']]
        pivot_rating = X_matrix_data.loc[u,m]
        pred_rating = X_hat_df.loc[u,m]

        column_movie = X_matrix_data.loc[:,m]
        total_count = X_matrix_data.loc[:,m].value_counts().sum()
        zero_count = np.count_nonzero(column_movie.values==0)

        if total_count-zero_count != 0:
          avg_rating = X_matrix_data.loc[:,m].sum() / (total_count-zero_count)
        else:
          avg_rating = 0.0
          
        analysis.append([u, m, true_rating, pred_rating, pivot_rating, avg_rating])

    analysis_df = pd.DataFrame(analysis)
    analysis_df.columns = ['userId', 'movieId', 'true rating', 'predicted rating', 'matrix_rating', 'average_baseline_rating']
    
    print("Saving to: " + './ALS_test/'+f_name[f_num][:-4]+'.csv')
    analysis_df.to_csv('./ALS_test/'+f_name[f_num][:-4]+'.csv')

Saving to: ./ALS_test/k=2_lambda=1.csv
Saving to: ./ALS_test/k=2_lambda=0.1.csv
Saving to: ./ALS_test/k=2_lambda=0.01.csv
Saving to: ./ALS_test/k=5_lambda=1.csv
Saving to: ./ALS_test/k=5_lambda=0.1.csv
Saving to: ./ALS_test/k=5_lambda=0.01.csv
Saving to: ./ALS_test/k=10_lambda=1.csv
Saving to: ./ALS_test/k=10_lambda=0.1.csv
Saving to: ./ALS_test/k=10_lambda=0.01.csv
Saving to: ./ALS_test/k=15_lambda=1.csv
Saving to: ./ALS_test/k=15_lambda=0.1.csv
Saving to: ./ALS_test/k=15_lambda=0.01.csv
Saving to: ./ALS_test/k=20_lambda=1.csv


FileNotFoundError: ignored

In [ ]:
test_csv_files = [
                  './ALS_test/k=2_lambda=1.csv',
                  './ALS_test/k=2_lambda=0.1.csv',
                  './ALS_test/k=2_lambda=0.01.csv',
                  './ALS_test/k=5_lambda=1.csv',
                  './ALS_test/k=5_lambda=0.1.csv',
                  './ALS_test/k=5_lambda=0.01.csv',
                  './ALS_test/k=10_lambda=1.csv',
                  './ALS_test/k=10_lambda=0.1.csv',
                  './ALS_test/k=10_lambda=0.01.csv',
                  './ALS_test/k=15_lambda=1.csv',
                  './ALS_test/k=15_lambda=0.1.csv',
                  './ALS_test/k=15_lambda=0.01.csv',
                  './ALS_test/k=20_lambda=1.csv'
                  ]

In [ ]:
for test in test_csv_files:
    test_df = pd.read_csv(test)
    print(test[11:-4])
    print("RMSE - True vs Predicted: ", round(mean_squared_error(test_df['true rating'], test_df['predicted rating']), 2))
    print("RMSE - True vs Baseline: ", round(mean_squared_error(test_df['true rating'], test_df['average_baseline_rating']),2))

k=2_lambda=1
RMSE - True vs Predicted:  2.73
RMSE - True vs Baseline:  0.91
k=2_lambda=0.1
RMSE - True vs Predicted:  2.78
RMSE - True vs Baseline:  0.91
k=2_lambda=0.01
RMSE - True vs Predicted:  2.82
RMSE - True vs Baseline:  0.91
k=5_lambda=1
RMSE - True vs Predicted:  4.38
RMSE - True vs Baseline:  0.91
k=5_lambda=0.1
RMSE - True vs Predicted:  4.61
RMSE - True vs Baseline:  0.91
k=5_lambda=0.01
RMSE - True vs Predicted:  4.71
RMSE - True vs Baseline:  0.91
k=10_lambda=1
RMSE - True vs Predicted:  6.55
RMSE - True vs Baseline:  0.91
k=10_lambda=0.1
RMSE - True vs Predicted:  7.1
RMSE - True vs Baseline:  0.91
k=10_lambda=0.01
RMSE - True vs Predicted:  7.13
RMSE - True vs Baseline:  0.91
k=15_lambda=1
RMSE - True vs Predicted:  8.3
RMSE - True vs Baseline:  0.91
k=15_lambda=0.1
RMSE - True vs Predicted:  3.11
RMSE - True vs Baseline:  0.91
k=15_lambda=0.01
RMSE - True vs Predicted:  3.07
RMSE - True vs Baseline:  0.91
k=20_lambda=1
RMSE - True vs Predicted:  3.21
RMSE - True vs Bas

In [ ]:
for test in test_csv_files:
    test_df = pd.read_csv(test)
    print(test[11:-4])
    print("RMSE - True vs Predicted: ", round(mean_squared_error(test_df['true rating'], test_df['predicted rating']), 2))
    print("RMSE - True vs Baseline: ", round(mean_squared_error(test_df['true rating'], test_df['average_baseline_rating']),2))